# Vraag 2 - Opdracht 3
 Bouw een systeem waar je anomalies detecteert aan de hand van: GAN (Generative Adversarial Network).

## Importeer benodigde bibliotheken

In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, multiply, Embedding
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

## Laad en bescrhijf de dataset

In [ ]:
# Laad de dataset
# Voeg uw bestandspad naar 'creditcard.csv' toe
data = pd.read_csv('creditcard.csv')
data.drop("Time", 1, inplace=True)
data.describe()

<ipython-input-27-541e47125df5>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop("Time", 1, inplace=True)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,245601.000000,...,245601.000000,245600.000000,245600.000000,245600.000000,245600.000000,245600.000000,245600.000000,245600.000000,245600.000000,245600.000000
mean,-0.043878,-0.019547,0.114126,0.029957,-0.041684,0.020474,-0.016718,0.001921,0.001093,-0.001357,...,-0.002872,-0.013349,-0.007133,0.000727,0.024349,0.001784,-0.000913,0.000992,91.033723,0.001828
std,1.961548,1.672675,1.505940,1.415156,1.376978,1.325868,1.228648,1.198433,1.114458,1.089061,...,0.739842,0.715181,0.640215,0.606488,0.514354,0.485980,0.402632,0.334605,252.602241,0.042718
min,-56.407510,-72.715728,-33.680984,-5.683171,-42.147898,-26.160506,-43.557242,-73.216718,-13.434066,-24.588262,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-11.710896,0.000000,0.000000
25%,-0.939484,-0.607357,-0.693656,-0.833469,-0.730162,-0.745197,-0.564340,-0.200834,-0.656634,-0.526248,...,-0.225626,-0.534212,-0.165440,-0.350210,-0.288756,-0.330334,-0.070521,-0.050077,6.000000,0.000000
50%,-0.009005,0.062184,0.305818,0.019751,-0.099435,-0.248624,0.019483,0.029589,-0.060559,-0.092146,...,-0.031992,-0.003622,-0.019048,0.045625,0.056638,-0.058891,0.002425,0.014284,23.320000,0.000000
75%,1.259759,0.781399,1.109678,0.803801,0.560701,0.422954,0.545423,0.330343,0.608041,0.456778,...,0.175963,0.493333,0.134211,0.431072,0.366624,0.248898,0.088917,0.077757,79.970000,0.000000
max,2.454930,22.057729,9.382558,16.875344,34.801666,22.529298,36.877368,20.007208,15.594995,23.745136,...,27.202839,10.503090,22.528412,4.022866,7.519589,3.517346,12.152401,33.847808,19656.530000,1.000000


Controleer op nullen

In [ ]:
data.isnull().sum().sum()

9

In [ ]:
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class'],
      dtype='object')

Checking duplicate rows

In [ ]:
# checking for duplicate values
print(f"Dataset has {data.duplicated().sum()} duplicate rows")
# dropping duplicate rows
data.drop_duplicates(inplace=True)

Dataset has 7329 duplicate rows


De enige niet vervormde variablen zijn:
- Time
- Amount
- Class (1: fraude & 0: niet fraude)

In [ ]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

,Target,Count,percent
0,0.0,237841,99.82
1,1.0,430,0.18


## Data Voorbereiden

In [ ]:
scaler = StandardScaler()
#scaler = MinMaxScaler()
X = scaler.fit_transform(data.drop('Class', 1))
y = data['Class'].fillna(0)


X= np.nan_to_num(X)

print(X.shape, y.shape)

<ipython-input-32-09d771733640>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = scaler.fit_transform(data.drop('Class', 1))


(238272, 29) (238272,)


In [ ]:
type(X)

numpy.ndarray

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

SMOTE Upsampling

In [ ]:
from imblearn.over_sampling import SMOTE
# training with best weights
pipe = Pipeline([
        ('smote', SMOTE(sampling_strategy= 0.2)),
        ('lr', LogisticRegression(class_weight={0: 0.95, 1: 0.05}))
])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [ ]:
# evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     23785
         1.0       0.90      0.86      0.88        43

    accuracy                           1.00     23828
   macro avg       0.95      0.93      0.94     23828
weighted avg       1.00      1.00      1.00     23828



## GAN Model

In [ ]:
class cGAN():
    def __init__(self):
        self.latent_dim = 32
        self.out_shape = 29
        self.num_classes = 2
        self.clip_value = 0.01
        #optimizer = RMSprop(lr=0.00005)

        # build discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=Adam(0.0002, 0.5),
                                   metrics=['accuracy'])

        # build generator
        self.generator = self.build_generator()

        # generating new data samples
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        gen_samples = self.generator([noise, label])

        self.discriminator.trainable = False

        # passing gen samples through disc.
        valid = self.discriminator([gen_samples, label])

        # combining both models
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
                              optimizer=Adam(0.0002, 0.5),
                             metrics=['accuracy'])
        self.combined.summary()

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(128, input_dim=self.latent_dim))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(256))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(self.out_shape, activation='tanh'))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        gen_sample = model(model_input)

        return Model([noise, label], gen_sample, name="Generator")


    def build_discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(512, input_dim=self.out_shape, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(128, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        gen_sample = Input(shape=(self.out_shape,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.out_shape)(label))

        model_input = multiply([gen_sample, label_embedding])
        validity = model(model_input)

        return Model(inputs=[gen_sample, label], outputs=validity, name="Discriminator")


    def train(self, X_train, y_train, pos_index, neg_index, epochs, batch_size=32, sample_interval=50):

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            #  Train Discriminator with 8 sample from postivite class and rest with negative class
            idx1 = np.random.choice(pos_index, 8)
            idx0 = np.random.choice(neg_index, batch_size-8)
            idx = np.concatenate((idx1, idx0))
            samples, labels = X_train[idx], y_train[idx]
            samples, labels = shuffle(samples, labels)
            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a half batch of new images
            gen_samples = self.generator.predict([noise, labels])

            # label smoothing
            if epoch < epochs//1.5:
                valid_smooth = (valid+0.1)-(np.random.random(valid.shape)*0.1)
                fake_smooth = (fake-0.1)+(np.random.random(fake.shape)*0.1)
            else:
                valid_smooth = valid
                fake_smooth = fake

            # Train the discriminator
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch([samples, labels], valid_smooth)
            d_loss_fake = self.discriminator.train_on_batch([gen_samples, labels], fake_smooth)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            # Condition on labels
            self.discriminator.trainable = False
            sampled_labels = np.random.randint(0, 2, batch_size).reshape(-1, 1)
            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            if (epoch+1)%sample_interval==0:
                print (f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

In [ ]:
type(y_train)

pandas.core.series.Series

Initieer en Train GAN

In [ ]:
cgan = cGAN()


y_train = y_train.values.reshape(-1,1)
pos_index = np.where(y_train==1)[0]
neg_index = np.where(y_train==0)[0]
cgan.train(X_train, y_train, pos_index, neg_index, epochs=20)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 512)               15360     
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_49 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_37 (LeakyReLU)  (None, 256)               0         
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_50 (Dense)            (None, 128)               32896     
                                                                 
 leaky_re_lu_38 (LeakyReLU)  (None, 128)             

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 128)               4224      
                                                                 
 leaky_re_lu_39 (LeakyReLU)  (None, 128)               0         
                                                                 
 batch_normalization_18 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 dense_53 (Dense)            (None, 256)               33024     
                                                                 
 leaky_re_lu_40 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_19 (Ba  (None, 256)               1024      
 tchNormalization)                                   

Genereer nieuwe data

In [ ]:
noise = np.random.normal(0, 1, (400, cgan.latent_dim))
sampled_labels = np.ones(400).reshape(-1, 1)

gen_samples = cgan.generator.predict([noise, sampled_labels])
gen_samples = scaler.inverse_transform(gen_samples)
print(gen_samples.shape)

13/13 [==============================] - 0s 2ms/step
(400, 29)


In [ ]:
gen_df = pd.DataFrame(data = gen_samples, columns = data.drop('Amount',1).columns)
gen_df.head()

<ipython-input-41-a4ec82b9edf0>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gen_df = pd.DataFrame(data = gen_samples, columns = data.drop('Amount',1).columns)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-0.758052,1.288144,-1.342018,1.232986,0.273459,-0.349000,-0.848788,0.262969,-0.372672,-0.540497,...,0.236125,-0.695626,0.312610,0.276764,0.415742,0.386376,0.286454,0.244437,-0.315637,180.465134
1,-1.465973,0.948576,-1.269252,1.397392,0.850078,-0.945952,-0.643894,-0.146435,0.832508,-0.877423,...,0.358971,-0.625435,0.004463,0.399842,-0.233616,0.502217,0.393577,0.067456,-0.257450,35.967640
2,-1.300375,1.412769,-1.206422,1.232640,0.960009,-0.915970,-0.917909,0.438297,-0.156996,-0.829954,...,0.538926,-0.542470,0.444975,0.325448,0.369763,0.017100,0.340830,0.303314,-0.321612,125.264435
3,-1.031813,0.915283,-1.259857,1.165901,0.675090,-0.357031,-1.025265,-0.044445,-0.103511,-0.748608,...,0.542392,-0.642467,0.199345,-0.158608,0.103445,0.493351,0.341569,0.212711,-0.327550,112.872383
4,-1.506379,0.328978,-1.314521,1.284158,0.007904,-0.658850,-0.531435,0.671558,0.124763,-0.665979,...,0.400990,-0.597087,-0.002886,0.031194,0.234043,0.255144,0.430518,0.077590,-0.319988,6.143619


Data voorbereiden voor anomalie detectie

In [ ]:
gen_df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Class'],
      dtype='object')

In [ ]:
gen_df['Class'].value_counts()

 180.465134    1
 61.683369     1
 193.278503    1
 28.147938     1
 47.147034     1
              ..
 9.022265      1
-4.165097      1
 73.378555     1
 194.133926    1
-13.716100     1
Name: Class, Length: 400, dtype: int64

Train model

In [ ]:
# Splitsen van kenmerken en labels
Xa = gen_df.drop(['Class'], axis=1).values
ya = gen_df["Class"].values



# Splitsen van de dataset in trainings- en validatiesets
train_x, val_x, train_y, val_y = train_test_split(Xa, ya, test_size=0.25)

from sklearn.preprocessing import StandardScaler


from sklearn.tree import DecisionTreeClassifier

# Trainen van een logistisch regressiemodel voor classificatie
clf = LogisticRegression().fit(train_x, train_y)
pred_y = clf.predict(val_x)


ValueError: ignored

detect anmomalies op gegenereerde data

In [ ]:
# Evaluatie
from sklearn.metrics import accuracy_score
# Functie om evaluatiemetrieken af te drukken
def print_test_score(label, prediction):
    print("Test Resultaat:\n================================================")
    print(f"Nauwkeurigheids Score: {accuracy_score(label, prediction) * 100:.2f}%")
    print("_______________________________________________")
    print(f"Classificatie Rapport:\n{classification_report(label, prediction)}")
    print("_______________________________________________")
    print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n")

# Evalueer het model op de testset
print_test_score(val_y, pred_y)

# ware/verkeerde positieven/negatieven
(tn, fp,
 fn, tp) = confusion_matrix(val_y, pred_y).flatten()

# Berekenen van precisie en recall
print(f"""
% van transacties gelabeld als fraude die correct waren (precisie): {tp}/({fp}+{tp}) = {tp/(fp+tp):.2%}
% van succesvol gevangen frauduleuze transacties (recall):    {tp}/({fn}+{tp}) = {tp/(fn+tp):.2%}
""")

---

## Confusion Matrix reports:
`[[TN, FP] `

**True Negatives (TN):** Dit zijn de gevallen waarin uw model correct voorspelt dat een transactie normaal is.

**False Positives (FP):** Dit zijn de gevallen waarin uw model onterecht voorspelt dat een normale transactie frauduleus is.


`[FN, TP]]`

**False Negatives (FN):** Dit zijn de frauduleuze transacties die uw model niet heeft gedetecteerd.


**True Positives (TP):** Dit zijn de gevallen waarin uw model correct voorspelt dat de transactie frauduleus is.

---

## Classification Report:
Het classificatierapport bevat verschillende belangrijke statistieken:

**Precision**: Geeft aan hoeveel van de door het model als positief (frauduleus) geclassificeerde gevallen daadwerkelijk positief zijn.

**Recall**: Geeft aan hoeveel van de werkelijke positieve gevallen het model correct heeft geïdentificeerd.

**F1-score**: De harmonische gemiddelde van precisie en recall, een balans tussen de twee.

**Support**: Het aantal werkelijke gevallen voor elke klasse in de dataset.

**Accuracy**: Het totale percentage correcte voorspellingen door het model.

**Macro avg**: Het ongewogen gemiddelde van de statistieken voor elke klasse.

**Weighted avg**: Het gewogen gemiddelde van de statistieken voor elke klasse, waarbij de gewichten de ondersteuning (aantal gevallen) voor elke klasse zijn.
